In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import datetime
import tempfile
import json
import os
from pathlib import Path as Pathlib
import yaml
# CSS styles for dark mode
style = """
<style>
    .widget-label { font-size: 16px; color: black; } /* Adjust the font size and color of labels for dark mode */
    .widget-radio-box label { /* Styling for radio button labels in dark mode */
        color: black; /* Change text color to white for better visibility */
    }
</style>
"""
display(HTML(style))

with open("./experiments/experiment_cfg.yaml", "r") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)
task_mapping = cfg["task_mapping"]
model_mapping = cfg["model_mapping"]
result_mapping = cfg["result_mapping"]

selection_display = widgets.HTML()
def update_display(*args):
    task = task_mapping[task_radio.value]
    model = model_mapping[model_radio.value]
    result = result_mapping[result_radio.value]

    with open("./experiments/run_time.json", "r") as f:
        data = json.load(f)
        task = data["task"]
        model = data["model"]
        runtime_folder = data["runtime_folder"]

    html_content = f"""
    <div class="selection-display">
        <p>Task Selected: {task_radio.value}</p>
        <p>Model Selected: {model_radio.value}</p>
        <p>Data written to temporary file: {f.name}</p>
        <p>Runtime folder: {runtime_folder}</p>
    </div>
    """
    selection_display.value = html_content

# Radio buttons for each category
task_radio = widgets.RadioButtons(options=list(task_mapping.keys()), description='Task Name:', disabled=False)
model_radio = widgets.RadioButtons(options=list(model_mapping.keys()), description='Model Name:', disabled=False)
result_radio = widgets.RadioButtons(options=list(result_mapping.keys()), description='Result:', disabled=False)

# Button to submit the choice
button = widgets.Button(description="Record Experiment")

# Output area for confirming the recorded data
output = widgets.Output()

# Function to handle button click
def on_button_clicked(b):
    with output:
        output.clear_output()

        task = task_mapping[task_radio.value]
        model = model_mapping[model_radio.value]
        now = datetime.datetime.now()

        experiment_parent_folder = f"./experiments/{task}/{model}"

        os.makedirs(experiment_parent_folder, exist_ok=True)

        experiment_id = 0
        for path in Pathlib(experiment_parent_folder).glob(f"run*"):
            if not path.is_dir():
                continue
            try:
                run_id = int(path.name.split("_")[-1])
                if run_id > experiment_id:
                    experiment_id = run_id
            except BaseException:
                pass
        experiment_id += 1
        
        experiment_folder = f"{experiment_parent_folder}/run_{experiment_id}"
        os.makedirs(experiment_folder, exist_ok=True)
        # Displaying the recorded information
        print(f"Recorded at {now}: Task: {task}, Model: {model}")
        
        # Writing to a temporary file
        with open("./experiments/run_time.json", "w") as f:
            json.dump({"task": task, 
                       "model": model,
                       "runtime_folder": experiment_folder}, f)
            print(f"Data written to temporary file: {f.name}")

        update_display()

task_radio.observe(update_display, 'value')
model_radio.observe(update_display, 'value')

create_button = widgets.Button(description="Create Experiment")
create_button.on_click(on_button_clicked)

# Initial display update
update_display()

# Display the widgets
display(task_radio, model_radio, selection_display, create_button)


In [ ]:
import plotly.express as px
import pandas as pd
# CSS styles for dark mode
style = """
<style>
    .widget-label { font-size: 16px; color: black; } /* Adjust the font size and color of labels for dark mode */
    .widget-radio-box label { /* Styling for radio button labels in dark mode */
        color: black; /* Change text color to white for better visibility */
    }
</style>
"""
display(HTML(style))

# Radio buttons for each category
result_radio = widgets.RadioButtons(options=list(result_mapping.keys()), description='Result:', disabled=False)

# Button to submit the choice
button = widgets.Button(description="Record Experiment")

# Output area for confirming the recorded data
plot_output = widgets.Output()


# Function to handle button click
def on_result_button_clicked(b):
    with open("./experiments/run_time.json", "r") as f:
        data = json.load(f)
        task = data["task"]
        model = data["model"]

    experiment_name = f"{task}_{model}"
    folder_path = f"./experiments/{experiment_name}"
    os.makedirs(folder_path, exist_ok=True)
    # with output:
    #     output.clear_output()

    task = task_mapping[task_radio.value]
    model = model_mapping[model_radio.value]
    result = result_mapping[result_radio.value]
    now = datetime.datetime.now()
    
    # Displaying the recorded information
    print(f"Recorded at {now}: Task: {task}, Model: {model}, Result: {result}")
    
    with open("./experiments/run_time.json", "r") as f:
        data = json.load(f)
        runtime_folder = data["runtime_folder"]

    print(runtime_folder)
    # Writing to a temporary file
    with open(os.path.join(runtime_folder, "results.json"), "w") as f:
        json.dump({"result": result}, f)
        if result == "vision_failure":
            # we need to create a file so that the result can be properly read.
            with open(os.path.join(runtime_folder, "finish.txt"), "w") as f2:
                pass
    update_visualization()

def update_visualization(*args):
    with plot_output:
        plot_output.clear_output(wait=True)  # Clear the previous output including Plotly figure
        results_file = "experiments/results.json"
        with open("experiments/run_time.json", "r") as f:
            data = json.load(f)
            runtime_folder = data["runtime_folder"]
            task_name = data["task"]
            model_name = data["model"]
        parent_folder = os.path.dirname(os.path.dirname(runtime_folder))
        results = {}

        for path in Pathlib(parent_folder).glob(f"*"):
            if not path.is_dir():
                continue
            model_name = str(path).split("/")[-1]
            if model_name not in results:
                results[model_name] = {}
            for path in Pathlib(parent_folder).glob(f"{model_name}/run*"):
                if not path.is_dir():
                    continue
                if not os.path.exists(os.path.join(path, "finish.txt")):
                    continue
                result_file = os.path.join(path, "results.json")
                if os.path.exists(result_file):
                    with open(result_file, "r") as f:
                        data = json.load(f)
                        result = data["result"]
                        if result not in results[model_name]:
                            results[model_name][result] = 0
                        results[model_name][result] += 1

        print(results)
        # result is in a dict like this: {'orion': {'failed_to_complete_subgoal_2': 1}}. Visualize using plotly

        df = pd.DataFrame(results)
        df = df.transpose()
        df = df.fillna(0)
        df = df.astype(int)
        df = df.reset_index()
        df = df.rename(columns={"index": "model"})
        df = df.melt(id_vars=["model"], var_name="result", value_name="count")
        print(df)
        fig = px.bar(df, x="model", y="count", color="result", barmode="group")
        fig.show()

    
    # fig = px.bar(df, color="result", barmode="group")
    # fig.show()

record_button = widgets.Button(description="Record Result")
record_button.on_click(on_result_button_clicked)
# Initial display update

# Display the widgets
display(result_radio, record_button)

# update_visualization()


display(plot_output)
